In [79]:
import pandas as pd
import networkx as nx
from helpers import *
import pickle

# Data Loading

In [80]:
# Articles is the name of all the articles in our dataset
# the index serves also as an id to retrieve articles in other files
# the names are url encoded
articles_df = pd.read_csv("./data/wikispeedia_paths-and-graph/articles.tsv",names = ["name"] ,skiprows=12,sep="\t")

#The categories of the articles, row index is the same as for the articles df
categories_df = pd.read_csv("./data/wikispeedia_paths-and-graph/categories.tsv",names = ["name","category"],skiprows=13, sep="\t")


#each row of of the links df contains the list of article names that an article links to
links_df = pd.read_csv("./data/wikispeedia_paths-and-graph/links.tsv",skiprows=12, names = ["from","to"],sep="\t")

#
shortest_path_distance = pd.read_csv("./data/wikispeedia_paths-and-graph/shortest-path-distance-matrix.txt",names=["distances"], skiprows=17)

#
paths_finished_df = pd.read_csv("./data/wikispeedia_paths-and-graph/paths_finished.tsv",names = ["hashedIpAddress","timestamp","durationInSec","path","rating"],skiprows=16, sep="\t")
#
paths_unfinished_df = pd.read_csv("./data/wikispeedia_paths-and-graph/paths_unfinished.tsv",names=["hashedIpAddress",   "timestamp ",  "durationInSec",  "path",   "target",   "type"],skiprows=17, sep="\t")


##### articles data frame

In [81]:
articles_df

,name
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in
1,%C3%85land
2,%C3%89douard_Manet
3,%C3%89ire
4,%C3%93engus_I_of_the_Picts
...,...
4599,Zionism
4600,Zirconium
4601,Zoroaster
4602,Zuid-Gelders


##### categories dataframe

In [82]:
categories_df

,name,category
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,subject.History.British_History.British_Histor...
1,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,subject.People.Historical_figures
2,%C3%85land,subject.Countries
3,%C3%85land,subject.Geography.European_Geography.European_...
4,%C3%89douard_Manet,subject.People.Artists
...,...,...
5199,Zirconium,subject.Science.Chemistry.Chemical_elements
5200,Zoroaster,subject.People.Religious_figures_and_leaders
5201,Zuid-Gelders,subject.Geography.European_Geography
5202,Zuid-Gelders,subject.Language_and_literature.Languages


##### Links dataframe

In [83]:
links_df

,from,to
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Bede
1,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Columba
2,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,D%C3%A1l_Riata
3,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Great_Britain
4,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Ireland
...,...,...
119877,Zulu,South_Africa
119878,Zulu,Swaziland
119879,Zulu,United_Kingdom
119880,Zulu,Zambia


##### Shortest path distances

In [84]:
shortest_path_distance

,distances
0,0_____33333325634333435_2433544334_3_422343544...
1,_0____22222325623232424_2422544324_3_312242544...
2,__0___33222425623232324_2333444433_3_422343434...
3,___0__33333325634233334_2433434333_2_423343433...
4,____0_22323335633332435_2433545434_3_423343544...
...,...
4599,______22222325622231424_1322544334_3_422232544...
4600,______33333434523232434_2332544324_3_323333544...
4601,______22222424522231434_2322545434_3_422232544...
4602,______33333436733342435_2433545444_3_523353544...


##### Finished Paths dataframe

In [85]:
paths_finished_df

,hashedIpAddress,timestamp,durationInSec,path,rating
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NaN
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0
2,415612e93584d30e,1349298640,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NaN
3,64dd5cd342e3780c,1265613925,37,14th_century;Renaissance;Ancient_Greece;Greece,NaN
4,015245d773376aab,1366730828,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3.0
...,...,...,...,...,...
51313,15a13a1d66ef5456,1349231015,66,Yagan;Ancient_Egypt;Civilization,NaN
51314,2ef7ac844cefda58,1300254138,165,Yagan;Folklore;Brothers_Grimm;<;19th_century;C...,3.0
51315,12863abb7887f890,1385095372,228,Yagan;Australia;England;France;United_States;T...,NaN
51316,19f8284371753362,1298792567,56,Yarralumla%2C_Australian_Capital_Territory;Aus...,1.0


##### Unfinished Paths dataframe

In [86]:
paths_unfinished_df

,hashedIpAddress,timestamp,durationInSec,path,target,type
0,2426091a53125110,1297054935,1804,Obi-Wan_Kenobi,Microsoft,timeout
1,26141fd878806294,1297055651,1805,Julius_Caesar,Caracas,timeout
2,2b015fb8181c48f2,1297090819,1818,Malawi;Democracy;Alexander_the_Great,First_Crusade,timeout
3,53a53bc244e08a6a,1297094761,49,Paraguay,Mount_St._Helens,restart
4,53a53bc244e08a6a,1297099105,1808,Paraguay;Bolivia,Mount_St._Helens,timeout
...,...,...,...,...,...,...
24870,109ed71f571d86e9,1389787605,180,Franz_Kafka;Tuberculosis;World_Health_Organiza...,Cholera,restart
24871,232f992e57d43e8d,1389787697,6,Modern_history,Hollandic,restart
24872,2e09a7224600a7cd,1389798400,1900,Computer_programming;Linguistics;Culture;Popul...,The_Beatles,timeout
24873,60af9e2138051b96,1389799481,1903,Jamaica;United_Kingdom;World_War_II;Battle_of_...,Alan_Turing,timeout


# Data Cleaning

### Cleaning categories

In [87]:
categories_grouped_df = categories_df.groupby(["name"]).agg(lambda x : list(x))
categories_grouped_df

,category
name,
%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,[subject.History.British_History.British_Histo...
%C3%85land,"[subject.Countries, subject.Geography.European..."
%C3%89douard_Manet,[subject.People.Artists]
%C3%89ire,"[subject.Countries, subject.Geography.European..."
%C3%93engus_I_of_the_Picts,[subject.History.British_History.British_Histo...
...,...
Zionism,"[subject.People.Political_People, subject.Reli..."
Zirconium,[subject.Science.Chemistry.Chemical_elements]
Zoroaster,[subject.People.Religious_figures_and_leaders]


We can see that not all the articles have corresponding categories, as we can see below

In [88]:
categories_joined_df = articles_df.join(categories_grouped_df,on="name",how="left")
categories_joined_df [categories_joined_df.category.isna()]

,name,category
1210,Directdebit,NaN
1231,Donation,NaN
1600,Friend_Directdebit,NaN
3253,Pikachu,NaN
3849,Sponsorship_Directdebit,NaN
4545,Wowpurchase,NaN


We will be using BERTopic, pretrained on the categories that we already have, to fill out the missing categories of these articles

### Played games paths

In [89]:
paths_finished_df

,hashedIpAddress,timestamp,durationInSec,path,rating
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NaN
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0
2,415612e93584d30e,1349298640,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NaN
3,64dd5cd342e3780c,1265613925,37,14th_century;Renaissance;Ancient_Greece;Greece,NaN
4,015245d773376aab,1366730828,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3.0
...,...,...,...,...,...
51313,15a13a1d66ef5456,1349231015,66,Yagan;Ancient_Egypt;Civilization,NaN
51314,2ef7ac844cefda58,1300254138,165,Yagan;Folklore;Brothers_Grimm;<;19th_century;C...,3.0
51315,12863abb7887f890,1385095372,228,Yagan;Australia;England;France;United_States;T...,NaN
51316,19f8284371753362,1298792567,56,Yarralumla%2C_Australian_Capital_Territory;Aus...,1.0


In [90]:
paths_unfinished_df

,hashedIpAddress,timestamp,durationInSec,path,target,type
0,2426091a53125110,1297054935,1804,Obi-Wan_Kenobi,Microsoft,timeout
1,26141fd878806294,1297055651,1805,Julius_Caesar,Caracas,timeout
2,2b015fb8181c48f2,1297090819,1818,Malawi;Democracy;Alexander_the_Great,First_Crusade,timeout
3,53a53bc244e08a6a,1297094761,49,Paraguay,Mount_St._Helens,restart
4,53a53bc244e08a6a,1297099105,1808,Paraguay;Bolivia,Mount_St._Helens,timeout
...,...,...,...,...,...,...
24870,109ed71f571d86e9,1389787605,180,Franz_Kafka;Tuberculosis;World_Health_Organiza...,Cholera,restart
24871,232f992e57d43e8d,1389787697,6,Modern_history,Hollandic,restart
24872,2e09a7224600a7cd,1389798400,1900,Computer_programming;Linguistics;Culture;Popul...,The_Beatles,timeout
24873,60af9e2138051b96,1389799481,1903,Jamaica;United_Kingdom;World_War_II;Battle_of_...,Alan_Turing,timeout


We can see that the path is formatted in a way we can't use efficiently, we will contruct the path as an array

In [91]:
paths_finished_df["path"] = paths_finished_df["path"].apply(lambda x: x.split(";"))
paths_unfinished_df["path"] = paths_unfinished_df["path"].apply(lambda x: x.split(";"))

We will also add a column containg the name of the article for the start and target paths for each game

In [92]:
paths_finished_df["start"] = paths_finished_df["path"].apply(lambda x: x[0])
paths_finished_df["target"] = paths_finished_df["path"].apply(lambda x: x[-1])

paths_unfinished_df["start"] = paths_unfinished_df["path"].apply(lambda x: x[0])

We also know that backs are also recorded in the path, these decisions that the user made could be interesting would be analyzed seprately, 
but for analysis, we will only focus on the final path that a player used to try to find the target.
To this end, we implemented a method (remove_backs, found in the helpers module) that would remove the "false" movements the player made

In [93]:
paths_finished_df["path"] = paths_finished_df["path"].apply(remove_backs)
paths_unfinished_df["path"] = paths_unfinished_df["path"].apply(remove_backs)

In [94]:
paths_unfinished_df['path_length'] = paths_unfinished_df['path'].apply(lambda x: len(x))

In [95]:
# We add a column for the path length on the finished paths
paths_finished_df["path_length"] = paths_finished_df["path"].apply(len)

In [96]:
paths_finished_df['time_per_article'] = paths_finished_df.apply(lambda x: x['durationInSec']/x['path_length'], axis=1)

In [97]:
paths_finished_df

,hashedIpAddress,timestamp,durationInSec,path,rating,start,target,path_length,time_per_article
0,6a3701d319fc3754,1297740409,166,"[14th_century, 15th_century, 16th_century, Pac...",NaN,14th_century,African_slave_trade,9,18.444444
1,3824310e536af032,1344753412,88,"[14th_century, Europe, Africa, Atlantic_slave_...",3.0,14th_century,African_slave_trade,5,17.600000
2,415612e93584d30e,1349298640,138,"[14th_century, Niger, Nigeria, British_Empire,...",NaN,14th_century,African_slave_trade,8,17.250000
3,64dd5cd342e3780c,1265613925,37,"[14th_century, Renaissance, Ancient_Greece, Gr...",NaN,14th_century,Greece,4,9.250000
4,015245d773376aab,1366730828,175,"[14th_century, Italy, Roman_Catholic_Church, H...",3.0,14th_century,John_F._Kennedy,7,25.000000
...,...,...,...,...,...,...,...,...,...
51313,15a13a1d66ef5456,1349231015,66,"[Yagan, Ancient_Egypt, Civilization]",NaN,Yagan,Civilization,3,22.000000
51314,2ef7ac844cefda58,1300254138,165,"[Yagan, Folklore, 19th_century, Charles_Dicken...",3.0,Yagan,Fiction,7,23.571429
51315,12863abb7887f890,1385095372,228,"[Yagan, Australia, England, France, United_Sta...",NaN,Yagan,U.S._Open_%28tennis%29,7,32.571429
51316,19f8284371753362,1298792567,56,"[Yarralumla%2C_Australian_Capital_Territory, A...",1.0,Yarralumla%2C_Australian_Capital_Territory,Abraham_Lincoln,4,14.000000


In [98]:
paths_unfinished_df

,hashedIpAddress,timestamp,durationInSec,path,target,type,start,path_length
0,2426091a53125110,1297054935,1804,[Obi-Wan_Kenobi],Microsoft,timeout,Obi-Wan_Kenobi,1
1,26141fd878806294,1297055651,1805,[Julius_Caesar],Caracas,timeout,Julius_Caesar,1
2,2b015fb8181c48f2,1297090819,1818,"[Malawi, Democracy, Alexander_the_Great]",First_Crusade,timeout,Malawi,3
3,53a53bc244e08a6a,1297094761,49,[Paraguay],Mount_St._Helens,restart,Paraguay,1
4,53a53bc244e08a6a,1297099105,1808,"[Paraguay, Bolivia]",Mount_St._Helens,timeout,Paraguay,2
...,...,...,...,...,...,...,...,...
24870,109ed71f571d86e9,1389787605,180,"[Franz_Kafka, Tuberculosis, World_Health_Organ...",Cholera,restart,Franz_Kafka,6
24871,232f992e57d43e8d,1389787697,6,[Modern_history],Hollandic,restart,Modern_history,1
24872,2e09a7224600a7cd,1389798400,1900,"[Computer_programming, Linguistics, Culture]",The_Beatles,timeout,Computer_programming,3
24873,60af9e2138051b96,1389799481,1903,"[Jamaica, United_Kingdom, World_War_II, Battle...",Alan_Turing,timeout,Jamaica,4


We also noticed that for the unfinished paths, a number of them were just unplayed games where players did not play at all
This behaviour was seen when there was no movements made by the player (i.e. no visited paths) and there was a timeout.
We decided to remove these games as they would not add any value to our analysis
We also have to note that there might be other games where players have started playing the game (i.e. visited some pages)
but then just decided to quit without exiting the game, so we decided to keep them as to not make assumptions about wether they quit the game or really did timeout on the game

In [99]:
paths_unfinished_df = paths_unfinished_df.drop(paths_unfinished_df[(paths_unfinished_df["durationInSec"]>=1800) & (paths_unfinished_df["path"].apply(lambda x: len(x)==1))].index)

In [100]:
paths_unfinished_df['time_per_article'] = paths_unfinished_df.apply(lambda x: x['durationInSec']/x['path_length'], axis=1)

In [101]:
paths_unfinished_df

,hashedIpAddress,timestamp,durationInSec,path,target,type,start,path_length,time_per_article
2,2b015fb8181c48f2,1297090819,1818,"[Malawi, Democracy, Alexander_the_Great]",First_Crusade,timeout,Malawi,3,606.000000
3,53a53bc244e08a6a,1297094761,49,[Paraguay],Mount_St._Helens,restart,Paraguay,1,49.000000
4,53a53bc244e08a6a,1297099105,1808,"[Paraguay, Bolivia]",Mount_St._Helens,timeout,Paraguay,2,904.000000
5,131600803df4895e,1297100557,2009,"[Agriculture, History_of_the_world, China, Yan...",Grand_Canal_of_China,timeout,Agriculture,4,502.250000
6,486bb79910fe9dd2,1297101660,1932,"[Mind, Christianity, Europe, Renaissance, Anci...",Scouting,timeout,Mind,7,276.000000
...,...,...,...,...,...,...,...,...,...
24870,109ed71f571d86e9,1389787605,180,"[Franz_Kafka, Tuberculosis, World_Health_Organ...",Cholera,restart,Franz_Kafka,6,30.000000
24871,232f992e57d43e8d,1389787697,6,[Modern_history],Hollandic,restart,Modern_history,1,6.000000
24872,2e09a7224600a7cd,1389798400,1900,"[Computer_programming, Linguistics, Culture]",The_Beatles,timeout,Computer_programming,3,633.333333
24873,60af9e2138051b96,1389799481,1903,"[Jamaica, United_Kingdom, World_War_II, Battle...",Alan_Turing,timeout,Jamaica,4,475.750000


# Data Construction

In [102]:
G_articles = nx.from_pandas_edgelist(links_df,source="from", target="to", create_using=nx.DiGraph())

We compute the metrics of the nodes using the graph and store them in a Dictionnary

In [103]:
closeness = nx.closeness_centrality(G_articles)
betweenness = nx.betweenness_centrality(G_articles)
pagerank = nx.pagerank(G_articles)

## Exploratory Data Analysis

We will first construct a column containing all the outer edges and the inner edges of an article

In [104]:
#Out going edges for an article
out_links_df = links_df.groupby(["from"]).agg(lambda x : list(x))
#Adding the articles where there is no outgoing edges
out_links_df = pd.merge(articles_df,out_links_df,left_on="name",right_on="from",how="left") #could be removed
out_links_df["to"]= out_links_df["to"].apply(lambda d: d if isinstance(d, list) else []) #could be removed

display(out_links_df)

#Incoming edges for an article
in_links_df = links_df.groupby(["to"]).agg(lambda x : list(x))
#Adding the articles where there is no incoming edges
in_links_df = pd.merge(articles_df,in_links_df,left_on="name",right_on="to",how="left") #could be removed
in_links_df["from"] = in_links_df["from"].apply(lambda d: d if isinstance(d, list) else []) #could be removed
display(in_links_df)

,name,to
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,"[Bede, Columba, D%C3%A1l_Riata, Great_Britain,..."
1,%C3%85land,"[20th_century, Baltic_Sea, Crimean_War, Curren..."
2,%C3%89douard_Manet,"[Absinthe, Beer, Claude_Monet, Diego_Vel%C3%A1..."
3,%C3%89ire,"[Canada, English_language, George_VI_of_the_Un..."
4,%C3%93engus_I_of_the_Picts,"[D%C3%A1l_Riata, Durham, England, Great_Britai..."
...,...,...
4599,Zionism,"[18th_century, 19th_century, Adolf_Hitler, Alb..."
4600,Zirconium,"[Aluminium, Arabic_language, Australia, Bicycl..."
4601,Zoroaster,"[18th_century, 9th_century, Afghanistan, Age_o..."
4602,Zuid-Gelders,"[Brabantian, Dutch_language, East_Flemish, Hol..."


,name,from
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,[]
1,%C3%85land,[]
2,%C3%89douard_Manet,[]
3,%C3%89ire,[]
4,%C3%93engus_I_of_the_Picts,[]
...,...,...
4599,Zionism,"[19th_century, Abraham_Goldfaden, Albert_Einst..."
4600,Zirconium,"[Calcium, Cerium, Diamond_simulant, Gas_metal_..."
4601,Zoroaster,"[Friedrich_Nietzsche, Greco-Buddhism, Iran, Ir..."
4602,Zuid-Gelders,"[Brabantian, Dutch_language, East_Flemish, Hol..."


In [105]:
out_links_df["num_links"]=out_links_df["to"].apply(lambda x: len(x))
out_links_df.sort_values(by="num_links",ascending=False).iloc[:10]

,name,to,num_links
4297,United_States,"[Abraham_Lincoln, Advertising, Agriculture, Am...",294
1247,Driving_on_the_left_or_right,"[Afghanistan, Africa, Akrotiri_and_Dhekelia, A...",255
2504,List_of_countries,"[Abkhazia, Afghanistan, Akrotiri_and_Dhekelia,...",244
2503,List_of_circulating_currencies,"[Abkhazia, Afghanistan, Akrotiri_and_Dhekelia,...",236
2515,List_of_sovereign_states,"[Abkhazia, Afghanistan, African_Union, Albania...",216
128,Africa,"[Abidjan, Abuja, Accra, Addis_Ababa, African_G...",212
2505,List_of_countries_by_system_of_government,"[Afghanistan, Albania, Algeria, Andorra, Angol...",207
2433,Lebanon,"[Afghanistan, Africa, Agriculture, Albania, Al...",192
2134,Interpol,"[Afghanistan, Albania, Algeria, Andorra, Angol...",191
340,Armenia,"[1st_century, Afghanistan, Agriculture, Albani...",186


We can see the top 10 nodes with the highest number of out-degrees are either pages for countries or listing countries

In [106]:
in_links_df["num_links"]=in_links_df["from"].apply(lambda x: len(x))
in_links_df.sort_values(by="num_links",ascending=False).iloc[:10]

,name,from,num_links
4297,United_States,"[%E2%82%AC2_commemorative_coins, 15th_Marine_E...",1551
4293,United_Kingdom,"[%C3%85land, %C3%89ire, %E2%82%AC2_commemorati...",972
1568,France,"[%C3%89douard_Manet, %E2%82%AC2_commemorative_...",959
1433,Europe,"[13th_century, 14th_century, 15th_century, 16t...",933
1385,England,"[%C3%93engus_I_of_the_Picts, 11th_century, 12t...",751
4542,World_War_II,"[%C3%85land, 15th_Marine_Expeditionary_Unit, 1...",751
1694,Germany,"[%C3%89douard_Manet, %E2%82%AC2_commemorative_...",743
2098,India,"[10th_century, 11th_century, 12th_century, 14t...",611
1389,English_language,"[%C3%89ire, A_Christmas_Carol, A_Wrinkle_in_Ti...",598
2538,London,"[1896_Summer_Olympics, 18th_century, 2005_Hert...",587


We can also see that most of the top 10 pages with the most incoming degrees are countries

Now we construct a dataframe with all relevant information we have so far.

In [107]:
nodes = pd.merge(in_links_df, out_links_df, on="name")
nodes = nodes.set_index(nodes['name'])
nodes["in_degree"] = nodes["to"].apply(lambda x : len(x) )
nodes["out_degree"] = nodes["from"].apply(lambda x : len(x) )
nodes["closeness"] = nodes.index.to_series().apply(lambda x :  closeness.get(x,-1) )
nodes["betweenness"] = nodes.index.to_series().apply(lambda x : betweenness.get(x,-1))
nodes["pagerank"] = nodes.index.to_series().apply(lambda x : pagerank.get(x,-1))
nodes = nodes.drop(["name","num_links_x","num_links_y"],axis=1)

In [108]:
display(nodes.sort_values(by="closeness",ascending=False))

,from,to,in_degree,out_degree,closeness,betweenness,pagerank
name,,,,,,,
United_States,"[%E2%82%AC2_commemorative_coins, 15th_Marine_E...","[Abraham_Lincoln, Advertising, Agriculture, Am...",294,1551,0.590353,0.094090,0.009566
Europe,"[13th_century, 14th_century, 15th_century, 16t...","[18th_century, Africa, Albania, Amsterdam, Anc...",159,933,0.542750,0.026992,0.006340
United_Kingdom,"[%C3%85land, %C3%89ire, %E2%82%AC2_commemorati...","[Acts_of_Union_1707, Adam_Smith, Airbus, Angli...",168,972,0.541466,0.042384,0.006235
France,"[%C3%89douard_Manet, %E2%82%AC2_commemorative_...","[AIDS, Airbus, Airbus_A380, Algeria, Andorra, ...",85,959,0.539805,0.012002,0.006423
Germany,"[%C3%89douard_Manet, %E2%82%AC2_commemorative_...","[Adolf_Hitler, Afghanistan, Agnosticism, Alban...",169,743,0.524285,0.019409,0.004824
...,...,...,...,...,...,...,...
Lone_Wolf_%28gamebooks%29,[],[],0,0,-1.000000,-1.000000,-1.000000
Wikipedia_Text_of_the_GNU_Free_Documentation_License,[],[],0,0,-1.000000,-1.000000,-1.000000
Color_Graphics_Adapter,[],[],0,0,-1.000000,-1.000000,-1.000000


In [109]:
#We can see that some articles are not connected to any other article (i.e: closeness/betweenness centrality=-1)
# which may be something to consider in our further anayslis
#that's why we assigned them the value -1 instead of default 0.

display(nodes[ (nodes["closeness"]== -1)|(nodes["betweenness"]== -1) ] )

,from,to,in_degree,out_degree,closeness,betweenness,pagerank
name,,,,,,,
Badugi,[],[],0,0,-1.0,-1.0,-1.0
Color_Graphics_Adapter,[],[],0,0,-1.0,-1.0,-1.0
Donation,[],[],0,0,-1.0,-1.0,-1.0
Douglas_DC-4,[],[],0,0,-1.0,-1.0,-1.0
Lone_Wolf_%28gamebooks%29,[],[],0,0,-1.0,-1.0,-1.0
Private_Peaceful,[],[],0,0,-1.0,-1.0,-1.0
Schatzki_ring,[],[],0,0,-1.0,-1.0,-1.0
Suikinkutsu,[],[],0,0,-1.0,-1.0,-1.0
Underground_%28stories%29,[],[],0,0,-1.0,-1.0,-1.0


### Now we will investigate the topics(categories) of the articles.

We construct a new Dataframe similar to Links but with Topics instead of Articles. This Dataframe will be used to create a Graph.

In [110]:
topics_df = pd.merge(links_df,categories_df, left_on="from",right_on="name")
topics_df = topics_df.rename(columns={'category': 'category_from' })[['to','category_from']]

topics_df = pd.merge(topics_df, categories_df , left_on = "to",right_on="name")
topics_df = topics_df.rename(columns={'category': 'category_to' })[['category_from','category_to']]
topics = topics_df.explode('category_from').explode('category_to').dropna()
#this will be used for general topics later
topics_general = topics.copy() 

topics = topics.groupby(by=["category_from","category_to"]).size().reset_index()
topics = topics.rename(columns= {0 : 'count'}) #we will use the count to have a weighted graph
#Now we will add weights to the edges. Each weight corresponds to the probability of taking that node.
topics_sum = topics.groupby(by="category_from").sum()
topics = pd.merge(topics,topics_sum,left_on='category_from',right_on='category_from')
topics = topics.rename(columns={"count_x":"count","count_y":"sum"})
topics["weight"] = topics["count"]/topics["sum"]

In [111]:
display(topics.sort_values(by="category_from"))

,category_from,category_to,count,sum,weight
0,subject.Art.Art,subject.Art.Art,122,1198,0.101836
72,subject.Art.Art,subject.People.Military_People,2,1198,0.001669
71,subject.Art.Art,subject.People.Mathematicians,6,1198,0.005008
70,subject.Art.Art,subject.People.Human_Scientists,1,1198,0.000835
69,subject.Art.Art,subject.People.Historical_figures,13,1198,0.010851
...,...,...,...,...,...
10150,subject.Science.Physics.The_Planets,subject.Geography.North_American_Geography,6,680,0.008824
10151,subject.Science.Physics.The_Planets,subject.Geography.Storms,4,680,0.005882
10152,subject.Science.Physics.The_Planets,subject.History.Ancient_History_Classical_Hist...,14,680,0.020588
10139,subject.Science.Physics.The_Planets,subject.Geography.Climate_and_the_Weather,31,680,0.045588


In [112]:
#We create the graph
G_topics = nx.from_pandas_edgelist(topics, source="category_from", target="category_to", edge_attr= "weight", create_using=nx.DiGraph() )

Now we will create DataFrame of Inward/Outward nodes for topics

In [113]:
in_topics_df = topics_df.explode("category_to").rename(columns={"category_to":"topic","category_from":"inward"}).drop_duplicates(subset=["topic","inward"])
in_topics_df = in_topics_df[["topic","inward"]]
in_topics_df = in_topics_df.groupby(by="topic").agg(list)
in_topics_df["num_links"] = in_topics_df["inward"].apply(len)

In [114]:
display(in_topics_df.sort_values(by="num_links",ascending=False))

,inward,num_links
topic,,
subject.Geography.European_Geography.European_Countries,[subject.History.British_History.British_Histo...,129
subject.Countries,[subject.History.British_History.British_Histo...,129
subject.Geography.North_American_Geography,"[subject.People.Artists, subject.History.Gener...",128
subject.Geography.European_Geography,"[subject.Countries, subject.Geography.European...",127
subject.Geography.Geography_of_Great_Britain,[subject.History.British_History.British_Histo...,127
...,...,...
subject.History.North_American_History,"[subject.History.General_history, subject.Peop...",27
subject.People.Sports_and_games_people,"[subject.History.General_history, subject.Peop...",26
subject.Everyday_life.Sports_teams,"[subject.Everyday_life.Sports_teams, subject.P...",22


In [115]:
out_topics_df = topics_df.explode("category_from").rename(columns={"category_from":"topic","category_to":"outward"}).drop_duplicates(subset=["topic","outward"])
out_topics_df = out_topics_df[["topic","outward"]]
out_topics_df = out_topics_df.groupby(by="topic").agg(list)
out_topics_df["num_links"] = out_topics_df["outward"].apply(len)

In [116]:
display(out_topics_df.sort_values(by="num_links",ascending=False))

,outward,num_links
topic,,
subject.Countries,"[subject.History.British_History, subject.Citi...",125
subject.History.General_history,[subject.History.British_History.British_Histo...,121
subject.Geography.Geography_of_Great_Britain,[subject.History.British_History.British_Histo...,120
subject.Geography.North_American_Geography,"[subject.History.British_History, subject.Citi...",118
subject.Geography.European_Geography.European_Countries,"[subject.History.British_History, subject.Citi...",117
...,...,...
subject.IT.Software,"[subject.Citizenship.Politics_and_government, ...",43
subject.Science.Biology.Organisms,"[subject.Geography.General_Geography, subject....",42
subject.IT.Cryptography,"[subject.Countries, subject.Geography.European...",35


### Note:
We noticed that the topics are too specific and for that reason we will create another simpler dataframe by taking only the last category 
(i.e: subject.Science.Physics.General_Physics -> General_Physics)

In [117]:
#General Topics
topics_general["category_from"] = topics_general["category_from"].apply(lambda x: reduce(str(x) ))
topics_general["category_to"] = topics_general["category_to"].apply(lambda x: reduce(str(x)) )
topics_general = topics_general.drop_duplicates(subset=['category_from','category_to']).dropna()

In [118]:
display(topics_general.sort_values(by="category_from"))

,category_from,category_to
99302,Actors_models_and_celebrities,General_Geography
104201,Actors_models_and_celebrities,Films
43086,Actors_models_and_celebrities,Monarchs_of_Great_Britain
154240,Actors_models_and_celebrities,Law
21503,Actors_models_and_celebrities,Languages
...,...,...
127149,Writers_and_critics,Archaeology
92579,Writers_and_critics,Musical_genres_styles_eras_and_events
71783,Writers_and_critics,Sports
124744,Writers_and_critics,Plants


In [119]:
G_general_topics = nx.from_pandas_edgelist(topics_general, source="category_from", target="category_to", create_using=nx.DiGraph() )

In [120]:
#create dictionnaries that have the values of closeness/betweenness centrality for each node of General topic
closeness_t = nx.closeness_centrality(G_general_topics)
betweenness_t = nx.betweenness_centrality(G_general_topics)
pagerank_t = nx.pagerank(G_general_topics)

In [121]:
#Creating DataFrame fo IN/OUT nodes for general topics
in_topics_general = topics_general.groupby(by="category_from").agg(list)
in_topics_general["count"] = in_topics_general["category_to"].apply(len)
in_topics_general =  in_topics_general.reset_index().rename(columns={"category_from":"topic"})

In [122]:
display(in_topics_general.sort_values(by="count",ascending=False))

,topic,category_to,count
34,Countries,"[British_History, Politics_and_government, Cou...",124
59,General_history,[British_History_1500_and_before_including_Rom...,120
62,Geography_of_Great_Britain,[British_History_1500_and_before_including_Rom...,119
90,North_American_Geography,"[British_History, Politics_and_government, Cou...",117
47,European_Countries,"[British_History, Politics_and_government, Cou...",116
...,...,...,...
43,Electricity_and_Electronics,"[Countries, European_Countries, North_American...",45
112,Software,"[Politics_and_government, Countries, European_...",43
93,Organisms,"[General_Geography, Currency, Countries, Europ...",42
35,Cryptography,"[Countries, European_Countries, Geography_of_G...",35


In [123]:
out_topics_general = topics_general.groupby(by="category_to").agg(list)
out_topics_general["count"] = out_topics_general["category_from"].apply(len)
out_topics_general =  out_topics_general.reset_index().rename(columns={"category_to":"topic"})
display(out_topics_general.sort_values(by="count",ascending=False))

,topic,category_from,count
34,Countries,[British_History_1500_and_before_including_Rom...,127
47,European_Countries,[British_History_1500_and_before_including_Rom...,127
90,North_American_Geography,"[Artists, General_history, Geology_and_geophys...",126
48,European_Geography,"[Countries, European_Countries, General_histor...",125
11,Asian_Countries,"[General_history, Geology_and_geophysics, Natu...",125
...,...,...,...
91,North_American_History,"[General_history, Engineers_and_inventors, Rai...",27
116,Sports_and_games_people,"[General_history, Sports_and_games_people, Act...",26
118,Sports_teams,"[Sports_teams, Sports_and_games_people, Geogra...",22
35,Cryptography,"[Mathematicians, World_War_II, Cryptography, G...",18


In [124]:
nodes_topics = out_topics_general.merge(in_topics_general,'left','topic')
nodes_topics = nodes_topics.drop(["count_y","count_x"],axis=1)
nodes_topics["in_degree"] = nodes_topics["category_to"].apply(lambda x : len(x) )
nodes_topics["out_degree"] = nodes_topics["category_from"].apply(lambda x : len(x) )
nodes_topics["closeness_centrality"] = nodes_topics.topic.apply(lambda x :  closeness_t.get(x,-1) )
nodes_topics["betweenness_centrality"] = nodes_topics.topic.apply(lambda x : betweenness_t.get(x,-1))
nodes_topics["pagerank_centrality"] = nodes_topics.topic.apply(lambda x : pagerank_t.get(x,-1))

In [125]:
nodes_topics.sort_values(by=["closeness_centrality","betweenness_centrality"],ascending=False).head(100)

,topic,category_from,category_to,in_degree,out_degree,closeness_centrality,betweenness_centrality,pagerank_centrality
34,Countries,[British_History_1500_and_before_including_Rom...,"[British_History, Politics_and_government, Cou...",124,127,1.000000,0.012558,0.011999
47,European_Countries,[British_History_1500_and_before_including_Rom...,"[British_History, Politics_and_government, Cou...",116,127,1.000000,0.010566,0.011999
90,North_American_Geography,"[Artists, General_history, Geology_and_geophys...","[British_History, Politics_and_government, Cou...",117,126,0.992126,0.010859,0.011874
62,Geography_of_Great_Britain,[British_History_1500_and_before_including_Rom...,[British_History_1500_and_before_including_Rom...,119,125,0.984375,0.011496,0.011742
48,European_Geography,"[Countries, European_Countries, General_histor...","[British_History, Politics_and_government, Cou...",112,125,0.984375,0.008967,0.011803
...,...,...,...,...,...,...,...,...
50,Evolution_and_reproduction,"[General_history, Birds, Philosophy, Mathemati...","[Countries, European_Countries, Geography_of_G...",75,59,0.649485,0.001884,0.006316
10,Artists,"[Artists, General_history, Plants, Performers_...","[Countries, European_Countries, Politics_and_g...",83,58,0.646154,0.001560,0.005940
82,Military_People,"[General_history, Military_History_and_War, Br...",[British_History_1500_and_before_including_Rom...,68,56,0.639594,0.000766,0.005833
20,Cartoons,"[General_history, Computer_and_Video_games, Fi...","[British_History, Politics_and_government, Cou...",88,53,0.630000,0.001800,0.005142


We now start looking at the actual games the players made.
We first inspect the top 10 visited nodes for the winners (without considering the start and target nodes).

In [126]:
#list of articles that were visited without the start and finish
paths_finished_df["path"].apply(lambda x : x[1:-1]).explode().value_counts().iloc[:10]

United_States     8567
Europe            4185
United_Kingdom    3665
Earth             3076
England           2937
Africa            2612
World_War_II      2117
North_America     1703
Germany           1602
Animal            1540
Name: path, dtype: int64

We can see that the most visited paths are a mix of countries, general concepts and historical events

Moving on to the losed games:

In [127]:
paths_unfinished_df["path"].apply(lambda x : x[1:-1]).explode().value_counts().iloc[:10]

United_States     2700
United_Kingdom    1016
Europe             959
England            813
Earth              790
Human              582
Africa             561
North_America      520
Animal             507
Mammal             455
Name: path, dtype: int64

we can see that we have a relatively close distribution to the most visited nodes in the winner paths,
this seems to hint that there might exist a general intuition and startegy
that people have while playing the game

We also inspect the top 10 most frequent games that were played, starting first with the winner games

In [128]:
paths_finished_df[["start","target","path"]].groupby(["start","target"]).count().sort_values(by="path",axis=0,ascending=False)[:10]

,,path
start,target,
Asteroid,Viking,1043
Brain,Telephone,1040
Theatre,Zebra,905
Pyramid,Bean,642
Batman,Wood,148
Bird,Great_white_shark,138
Batman,The_Holocaust,119
Bird,Adolf_Hitler,107
Beer,Sun,99


Then with the unfinished games

In [129]:
paths_unfinished_df[["start","target","path"]].groupby(["start","target"]).count().sort_values(by="path",axis=0,ascending=False)[:10]

path
start    target         
Pyramid  Bean        898
Brain    Telephone   896
Theatre  Zebra       805
Asteroid Viking      677
Batman   Wood         69
         Banana       53
Beer     Sun          41
Aircraft Google       40
Cat      Microsoft    40
Dog      Telephone    36

We can see that there exist games with a relatively high number of plays and these games are the same in both the winner and loser games.
This would allow us to have sufficient data to perform an analysis on these games.

In [130]:
paths_finished_df

,hashedIpAddress,timestamp,durationInSec,path,rating,start,target,path_length,time_per_article
0,6a3701d319fc3754,1297740409,166,"[14th_century, 15th_century, 16th_century, Pac...",NaN,14th_century,African_slave_trade,9,18.444444
1,3824310e536af032,1344753412,88,"[14th_century, Europe, Africa, Atlantic_slave_...",3.0,14th_century,African_slave_trade,5,17.600000
2,415612e93584d30e,1349298640,138,"[14th_century, Niger, Nigeria, British_Empire,...",NaN,14th_century,African_slave_trade,8,17.250000
3,64dd5cd342e3780c,1265613925,37,"[14th_century, Renaissance, Ancient_Greece, Gr...",NaN,14th_century,Greece,4,9.250000
4,015245d773376aab,1366730828,175,"[14th_century, Italy, Roman_Catholic_Church, H...",3.0,14th_century,John_F._Kennedy,7,25.000000
...,...,...,...,...,...,...,...,...,...
51313,15a13a1d66ef5456,1349231015,66,"[Yagan, Ancient_Egypt, Civilization]",NaN,Yagan,Civilization,3,22.000000
51314,2ef7ac844cefda58,1300254138,165,"[Yagan, Folklore, 19th_century, Charles_Dicken...",3.0,Yagan,Fiction,7,23.571429
51315,12863abb7887f890,1385095372,228,"[Yagan, Australia, England, France, United_Sta...",NaN,Yagan,U.S._Open_%28tennis%29,7,32.571429
51316,19f8284371753362,1298792567,56,"[Yarralumla%2C_Australian_Capital_Territory, A...",1.0,Yarralumla%2C_Australian_Capital_Territory,Abraham_Lincoln,4,14.000000


We add a column in the nodes_df for the number of times each article has been visited by a path (finished or unfinished)

In [131]:
nodes 

,from,to,in_degree,out_degree,closeness,betweenness,pagerank
name,,,,,,,
%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,[],"[Bede, Columba, D%C3%A1l_Riata, Great_Britain,...",11,0,0.000000,0.000000,0.000033
%C3%85land,[],"[20th_century, Baltic_Sea, Crimean_War, Curren...",19,0,0.000000,0.000000,0.000033
%C3%89douard_Manet,[],"[Absinthe, Beer, Claude_Monet, Diego_Vel%C3%A1...",20,0,0.000000,0.000000,0.000033
%C3%89ire,[],"[Canada, English_language, George_VI_of_the_Un...",8,0,0.000000,0.000000,0.000033
%C3%93engus_I_of_the_Picts,[],"[D%C3%A1l_Riata, Durham, England, Great_Britai...",10,0,0.000000,0.000000,0.000033
...,...,...,...,...,...,...,...
Zionism,"[19th_century, Abraham_Goldfaden, Albert_Einst...","[18th_century, 19th_century, Adolf_Hitler, Alb...",74,25,0.357160,0.000240,0.000206
Zirconium,"[Calcium, Cerium, Diamond_simulant, Gas_metal_...","[Aluminium, Arabic_language, Australia, Bicycl...",42,15,0.282427,0.000068,0.000088
Zoroaster,"[Friedrich_Nietzsche, Greco-Buddhism, Iran, Ir...","[18th_century, 9th_century, Afghanistan, Age_o...",28,9,0.331932,0.000056,0.000094


In [132]:
paths_finished_df

,hashedIpAddress,timestamp,durationInSec,path,rating,start,target,path_length,time_per_article
0,6a3701d319fc3754,1297740409,166,"[14th_century, 15th_century, 16th_century, Pac...",NaN,14th_century,African_slave_trade,9,18.444444
1,3824310e536af032,1344753412,88,"[14th_century, Europe, Africa, Atlantic_slave_...",3.0,14th_century,African_slave_trade,5,17.600000
2,415612e93584d30e,1349298640,138,"[14th_century, Niger, Nigeria, British_Empire,...",NaN,14th_century,African_slave_trade,8,17.250000
3,64dd5cd342e3780c,1265613925,37,"[14th_century, Renaissance, Ancient_Greece, Gr...",NaN,14th_century,Greece,4,9.250000
4,015245d773376aab,1366730828,175,"[14th_century, Italy, Roman_Catholic_Church, H...",3.0,14th_century,John_F._Kennedy,7,25.000000
...,...,...,...,...,...,...,...,...,...
51313,15a13a1d66ef5456,1349231015,66,"[Yagan, Ancient_Egypt, Civilization]",NaN,Yagan,Civilization,3,22.000000
51314,2ef7ac844cefda58,1300254138,165,"[Yagan, Folklore, 19th_century, Charles_Dicken...",3.0,Yagan,Fiction,7,23.571429
51315,12863abb7887f890,1385095372,228,"[Yagan, Australia, England, France, United_Sta...",NaN,Yagan,U.S._Open_%28tennis%29,7,32.571429
51316,19f8284371753362,1298792567,56,"[Yarralumla%2C_Australian_Capital_Territory, A...",1.0,Yarralumla%2C_Australian_Capital_Territory,Abraham_Lincoln,4,14.000000


In [133]:
# we count the number of times each article appears in the paths
all_paths_finished = paths_finished_df["path"].explode().sort_index().value_counts()
all_paths_unfinished = paths_unfinished_df["path"].explode().value_counts()
nodes['visits'] = nodes.index.map(all_paths_finished + all_paths_unfinished)
nodes = nodes.fillna(0)

# Exporting DATA

In [134]:
nodes_topics.to_pickle('data/df_nodes_topics.pkl')
nodes.to_pickle('data/df_nodes.pkl')
topics_general.to_pickle('data/df_topics_general.pkl')
out_topics_df.to_pickle('data/df_out_topics.pkl')
in_topics_df.to_pickle('data/df_in_topics_df.pkl')
out_topics_general.to_pickle('data/df_out_topics.pkl')
in_topics_general.to_pickle('data/df_in_topics.pkl')

nx.write_gpickle(G_articles,'data/G_articles.gpickle') #graph of nodes
nx.write_gpickle(G_topics,'data/G_topics.gpickle') #graph of topics
nx.write_gpickle(G_general_topics,'data/G_general_topics.gpickle') #graph of general topics


paths_finished_df.to_pickle('data/df_paths_finished.pkl')
paths_unfinished_df.to_pickle('data/df_paths_unfinished.pkl')
with open('data/closeness_dict.pkl','wb') as f:
    pickle.dump(closeness,f)
with open('data/closeness_topics_dict.pkl','wb') as f:
    pickle.dump(closeness_t,f)  

with open('data/pagerank_dict.pkl','wb') as f:
    pickle.dump(pagerank,f)  
with open('data/pagerank_topics_dict.pkl','wb') as f:
    pickle.dump(pagerank_t,f)  
    
with open('data/betweenness_dict.pkl','wb') as f:
    pickle.dump(betweenness,f)
with open('data/betweenness_topics_dict.pkl','wb') as f:
    pickle.dump(betweenness_t,f)      
    